In [12]:
from collections import OrderedDict

import numpy as np
import json

from bridge_base import ZkLoad
from bridge import Bridge_160
from checker import Checker

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 载入数据

In [7]:
# 常量
E = 21 * 10**10           # 杨氏模量 (Pa)
P = 1                     # 外力    (kN)
h = 2.52                  # 恒载    (kN)

bottom_chord_length = 8.   # 下弦杆长度/下节点间距 (m)

bridge_len = 160

# [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 40]
bottom_chord_nodes_nums = list(range(1, 40, 2)) + [40]

zk_load = ZkLoad(bridge_len)
path = './data/'

bridge = Bridge_160()
bridge.load_data(path)
bridge.load_params(E=E, P=P, h=h, 
                   bottom_chord_nodes_nums=bottom_chord_nodes_nums,
                   bottom_chord_length=bottom_chord_length, 
                   load=zk_load)


# 杆件分组文件
with open('units_clusters.json', 'r') as units_clusters_file:
    units_clusters = json.load(units_clusters_file)

In [14]:
# b = 2 * t1 + b2 = 0.46
# h = 2 * b1 + t2 = 0.76
B = {
    '0':  0.46, 
    '1':  0.46, 
    '2':  0.46, 
    '3':  0.46,
    '4':  0.46, 
    '5':  0.46, 
    '6':  0.46
}

T1 = {
    '0':  0.022,  
    '1':  0.022,  
    '2':  0.022, 
    '3':  0.022,
    '4':  0.022, 
    '5':  0.022, 
    '6':  0.022 
}


H = {
    '0':  0.60,  
    '1':  0.76,  
    '2':  0.44, 
    '3':  0.44,
    '4':  0.76, 
    '5':  0.60, 
    '6':  0.60 
}


T2 = {
    '0':  0.022,  
    '1':  0.022,  
    '2':  0.022, 
    '3':  0.022,
    '4':  0.022, 
    '5':  0.022, 
    '6':  0.022 
}

In [9]:
units_clusters

{'0': [2, 5, 6, 13, 17, 25, 26, 28, 30, 32, 40, 45, 50, 54, 58, 62, 66, 70],
 '1': [38, 42],
 '2': [1, 4, 8, 9, 21, 29, 34, 43, 47, 51, 55, 59, 63, 67, 71, 75],
 '3': [3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 52, 56, 60, 64],
 '4': [37, 46, 49, 57, 68, 69, 72, 76],
 '5': [10, 14, 18, 22, 33, 36, 41],
 '6': [12, 16, 20, 24, 44, 48, 53, 61, 65, 73, 74, 77]}

### 创建检算器

In [18]:
bridge.check()
checker = Checker(bridge)

### 更新截面参数

In [19]:
for label, units_cluster in units_clusters.items():
    # 每一类的参数
    b, t1 = B[label], T1[label]
    b2 = b - 2 * t1
    
    h, t2 = H[label], T2[label]
    b1 = (h - t2) / 2
    
    section_params = [b2, t1, t2, b1]
    bridge.set_section_params(units_cluster, section_params)          

bridge.update()

检算 => 所有单元 [单元刚度矩阵] 已重新计算.
检算 => 所有单元 [毛面积] 已重新计算.
检算 => [总体刚度矩阵] 已重新计算.
检算 => [缩减刚度矩阵] 已重新计算.
所有节点竖向位移已计算完毕.
所有杆件单元轴力已计算完毕.
所有零杆已标识.
所有杆件单元最不利荷载已计算完毕.
所有杆件单元最不利位移已计算完毕.


### 检算

In [23]:
checker.check_all()

AttributeError: 'Checker' object has no attribute 'h'